In [ ]:
import sys
sys.path.append('..')

from src.data import make_dataset
# from src.simulations import run_simulations
from tqdm import tqdm

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
word_list = []
with open("..\\data\\raw\\glove.6B.50d.txt", encoding="utf8") as f:
            for line in tqdm(f, desc="Reading file", unit="line", total=400_000):
                word, coefs = line.split(maxsplit=1)
                word_list += [word]

In [ ]:
print("len(words_array) (raw):\t\t\t" + "{:,}".format(len(word_list)))
word_list = make_dataset.filter_alphabetic(word_list)
print("len(words_array) (filter_alphabetic):\t" + "{:,}".format(len(word_list)))
word_list = make_dataset.filter_20k(word_list, "..\\data\\external\\20k.txt")
print("len(words_array) (filter_20k):\t\t" + "{:,}".format(len(word_list)))

In [ ]:
def get_wordnet_pos(word, tag_num=0):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][tag_num].lower()
    if tag in "nvars": return tag
    else: return "n"

def lemmatize_words(words):
    lemmatized_set = set()
    wnl = WordNetLemmatizer()

    for word in words:
        word_lemma_list = [wnl.lemmatize(word, pos=p) for p in "nvars"]
        shortest_lemma = min(word_lemma_list, key=len)
        if len(wordnet.synsets(shortest_lemma)) > 0:
            lemmatized_set.add(shortest_lemma)

    return list(lemmatized_set)

In [ ]:
lemma_list = lemmatize_words(word_list)
print(len(lemma_list))
np.random.choice(lemma_list, 20)

In [ ]:
np.random.choice(lemma_list, 20)

In [ ]:
wnl = WordNetLemmatizer()

In [ ]:
test_word = "incorporates"
print(nltk.pos_tag([test_word])[0])

print()
print("------ Double Lemma: Method 1 ------")
test_pos_1 = get_wordnet_pos(test_word)
lemma_1 = wnl.lemmatize(test_word, pos=test_pos_1)
test_pos_2 = get_wordnet_pos(lemma_1)
lemma_2 = wnl.lemmatize(lemma_1, pos=test_pos_2)

print("\ntest_word: " + test_word + 
      "\tpos: " + test_pos_1)
print("1st lemma: " + lemma_1 + 
      "\tpos: " + test_pos_2)
print("2nd lemma: " + lemma_2)

print()
print("------ Double Lemma: Method 2 ------")
test_pos_1 = get_wordnet_pos(test_word)
test_pos_2 = get_wordnet_pos(test_word, tag_num=1)
lemma_1 = wnl.lemmatize(test_word, pos=test_pos_1)
lemma_2 = wnl.lemmatize(lemma_1, pos=test_pos_2)

print("\ntest_word: " + test_word + 
      "\tpos: " + test_pos_1)
print("1st lemma: " + lemma_1 + 
      "\tpos: " + test_pos_2)
print("2nd lemma: " + lemma_2)

print()
print("------ Double Lemma: Method 3 ------")
test_lemma_list = [wnl.lemmatize(test_word, pos=p) for p in "nvars"]

print("\ntest_word: " + test_word)
for i, l in enumerate(test_lemma_list):
    print("pos: " + "nvars"[i] + "\tlemma: " + l)

print("selected lemma: " + min(test_lemma_list, key=len))

In [ ]:
embeddings_index = make_dataset.parse_glove_data("..\\data\\raw\\glove.6B.50d.txt", True)

In [ ]:
words_array = make_dataset.get_words_array(embeddings_index)
print("len(words_array) (raw):\t\t\t" + "{:,}".format(len(words_array)))
words_array = make_dataset.filter_alphabetic(words_array)
print("len(words_array) (filter_alphabetic):\t" + "{:,}".format(len(words_array)))
words_array = make_dataset.filter_20k(words_array, "..\\data\\external\\20k.txt")
print("len(words_array) (filter_20k):\t\t" + "{:,}".format(len(words_array)))


In [ ]:
glove_df = make_dataset.embeddings_to_dataframe(embeddings_index, words_array)